# Syntax: DSC execution

## DSC Sequences
Blocks are executed from the **DSC section** as *sequences*, defined by the `DSC::run` entry. Ways these blocks are connected can be specified via "*", "," and "()" operators:

_FIXME: give example for_ `+`

*  `*`: right hand side (RHS) is executed after the left hand side (LHS).
*  `+`: LHS and RHS are pair-wise combined first. **In DSC, + operator has higher precedence than * operator**.
*  `,`: RHS and LHS are concurrent.
*  `()`: allows for combinations of `*` and `,`.

These operators also apply to `.logic` entries which define the ways parameters are combined.

### Example 1
```yaml
  run: simulate * (transform * estimate) * score
```

DSC will run "transform" first, followed by "estimate".

### Example 2
```yaml
  run: simulate * (transform * estimate, estimate) * score
```

DSC will run 2 concurrent sequences of steps. The first involves "transform" followed by "estimate"; the second is "estimate" only.

Slicing is supported, for example:

```yaml
  run: simulate * (transform[1:3] * estimate[1], transform[4:5] * estimate[2], estimate) * score
```

### Example 3
```yaml
  run: simulate * (voom, sqrt, identity) * (RUV, SVA) * (DESeq, edgeRglm, ash) * score
```

will be expanded to:

```yaml
  run: simulate * sqrt * RUV * DESeq * score,
       simulate * sqrt * SVA * DESeq * score,
       simulate * identity * RUV * DESeq * score,
       simulate * voom * RUV * DESeq * score,
       simulate * identity * SVA * DESeq * score,
       simulate * voom * SVA * DESeq * score,
       simulate * sqrt * RUV * ash * score,
       simulate * sqrt * RUV * edgeRglm * score,
       simulate * sqrt * SVA * ash * score,
       simulate * sqrt * SVA * edgeRglm * score,
       simulate * identity * RUV * ash * score,
       simulate * voom * RUV * ash * score,
       simulate * identity * RUV * edgeRglm * score,
       simulate * voom * RUV * edgeRglm * score,
       simulate * identity * SVA * ash * score,
       simulate * voom * SVA * ash * score,
       simulate * identity * SVA * edgeRglm * score,
       simulate * voom * SVA * edgeRglm * score
```

## DSC Section Parameters
*  `master`: Block names that will become one of the *master* tables in DSC output. By default the last block of a DSC sequence will be the master table. When a block becomes a master table, DSC will attempt to extract information from the block output and write them explicitly to the DSC output database.

*FIXME: we should no longer need master parameter once we have simple interface to extract intermediate output*

*  `params`: Defines DSC variables which can be used anywhere in the DSC file through `$()` sigil.
*  `exec_path`: Search path for executable files.
*  `lib_path`: Search path for library files, such as R script to be `source`-ed or Python scripts to be `import`-ed.
*  `R_libs`: Required R libraries, from cran, bioconductor or github:
 *  `package`: DSC will check / install `package` from cran or bioconductor.
 *  `github_repo/package`: DSC will install `package` from `github_repo` on github.
 *  `package (version[s])`: DSC will install `package` and check version as required. A warning message will be generated if version do not match. It is possible to specify several versions such as `edgeR (3.12.0, 3.12.1)` or simply `edgeR (3.12.0+)`.
*  `python_modules`: Required Python3 modules, from pip.
*  `work_dir`: work directory where DSC runs take place.

## Running DSC
At this point we have covered everthing we need to compose a DSC file. To see how it works in practice with a simple example, please follow through this [Quick Start tutorial](../tutorials/Quick_Start.html).